In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
cd /content/drive/My Drive/Project_HonesDoctor2020

/content/drive/My Drive/Project_HonesDoctor2020


In [7]:
!pip install googletrans
!pip install tqdm #--upgrade
!pip install twython

  Created wheel for googletrans: filename=googletrans-2.4.0-cp36-none-any.whl size=15777 sha256=882f571cba4ea0971aea82f809eccfc115b79edbde1e306a77ef35d9e5c34e05
  Stored in directory: /root/.cache/pip/wheels/50/d6/e7/a8efd5f2427d5eb258070048718fa56ee5ac57fd6f53505f95
Successfully built googletrans


In [0]:
import requests
from bs4 import BeautifulSoup
from time import  time, sleep
from random import randint

from googletrans import Translator

from tqdm import tqdm
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from nltk import sent_tokenize, word_tokenize
from nltk.sentiment.vader import  SentimentIntensityAnalyzer
from nltk.sentiment.util import mark_negation

In [10]:
import nltk

nltk.download("punkt")
nltk.download("vader_lexicon")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [0]:
def honestdoc_comment(url):
  """
    INPUT
      url : String
        URL of the target website
    OUTPUT
      comment : List
        List of comments
      score : List
        List of rating score
  """
  data = requests.get(url)
  print("requests code : {}".format(data.status_code))
  print("note\n2xx : successs\n4xx, 5xx : error")

  start_time = time()
  r = requests.get(url, params=dict(query="web scraping", page=1))
  soup = BeautifulSoup(r.text, "html.parser")
  n = len(soup.find_all("div", {"class": "comments__content"}))

  comment = [ soup.find_all("div", {"class": "comments__content"})[i].get_text().strip() for i in range(0, n) ]
  score = [ soup.find_all("span", {"class": "stars star-rating"})[i].attrs["data-score"] for i in range(0, n) ]

  elapsed_time = time() - start_time
  print("Time used for scraping data from page 1 : {} s".format(elapsed_time))
  sleep(randint(1,3))

  p = 2
  while n > 0:
    start_time = time()
    r = requests.get(url, params=dict(query="web scraping", page=p))
    soup = BeautifulSoup(r.text, "html.parser")
    n = len(soup.find_all("div", {"class": "comments__content"}))
    [ comment.append(soup.find_all("div", {"class": "comments__content"})[i].get_text().strip()) for i in range(0,n) ]
    [ score.append(soup.find_all("span", {"class": "stars star-rating"})[i].attrs["data-score"]) for i in range(0,n) ]
    elapsed_time = time() - start_time
    print("Time used for scraping data from page {} : {} s".format(p, elapsed_time))
    p += 1
    sleep(randint(1,3))
  #backup
  pd.DataFrame({"comment": comment, "score": score}).to_csv("comment_"+str(url[url.rfind("/")+1:]) + ".csv", index=False)
  return comment, score


In [0]:
# We choose the following lists of hospitals to make sentiment analysis.
# โรงพยาบาลศิริราช ปิยมหาราชการุณย์
# https://www.honestdocs.co/hospitals/siriraj-piyamaharajkarun-hospital
# โรงพยาบาลสิรินธร
# https://www.honestdocs.co/hospitals/sirindhorn-hospital
# โรงพยาบาลตำรวจ
# https://www.honestdocs.co/hospitals/police-general-hospital

In [12]:
#โรงพยาบาลสิรินธร
sirin_comments, sirin_scores = honestdoc_comment(r"https://www.honestdocs.co/hospitals/sirindhorn-hospital")

requests code : 200
note
2xx : successs
4xx, 5xx : error
Time used for scraping data from page 1 : 1.2379305362701416 s
Time used for scraping data from page 2 : 1.2292325496673584 s
Time used for scraping data from page 3 : 1.2271668910980225 s
Time used for scraping data from page 4 : 1.225907564163208 s
Time used for scraping data from page 5 : 1.227487564086914 s
Time used for scraping data from page 6 : 1.2117831707000732 s
Time used for scraping data from page 7 : 1.3036956787109375 s
Time used for scraping data from page 8 : 1.2247300148010254 s
Time used for scraping data from page 9 : 1.2705328464508057 s
Time used for scraping data from page 10 : 1.2832105159759521 s
Time used for scraping data from page 11 : 1.2226510047912598 s
Time used for scraping data from page 12 : 1.2293052673339844 s
Time used for scraping data from page 13 : 1.2248027324676514 s
Time used for scraping data from page 14 : 1.2214021682739258 s
Time used for scraping data from page 15 : 1.2375061511993

In [13]:
sirin_comments

['แผนกฉุกเฉิน แผนกนรีเวชดีเยี่ยมากๆ และแผนกTBก็เยี่ยมมากๆ พนักงานธุรการไม่ค่อยโอเคเท่าไหร่แต่พยาบาลและคุณหมอดีมากๆ และดีทุกคนเลยค่ะ คุณหมอเก่งๆหลายท่านด้วยค่ะ ช่วงเช้าคนจะเยอะมากๆ อาจมีล่าช้าบ้างค่ะ แต่บางครั้งเราไป ไม่เกินเที่ยงก็ได้กลับบ้านแล้วนะคะ ถือว่าเร็วไวมากๆ ใช้บริการหลายครั้งค่อนข้างจะประทับใจค่ะ ถึงแม้จะใช้สิทธิ์ประกัน แต่ก็ได้รับการใส่ใจที่ดี ร้านอาหารร้านขนมมีให้ซื้อในราคาทั่วไปนะคะ ห้องน้ำชั้น2สะอาดดีค่ะ',
 'คนรอเยอะมากๆค่ะ (ไปตอน22.00 แผนกฉุกเฉิน) หมอคนเดียว \r\nพยาบาลพูดจาดีค่ะ \r\nให้บริการรักษาดี ถึงแม้จะมีหมอเพียงคนเดียว \r\nราคาไม่แพง\r\n\r\nต่อมาไปตามนัดหมอตอนเช้า 8.00 หมอมา10.30น. คนรอกว่า 30 คนค่ะ เรียกเข้าช้ามา ผ่านไป1ชั่วโมงเพิ่งผ่านไป6คิวค่ะ หากว่าสามารถมีแนวทางในการรักษาให้รวดเร็วขึ้นจะดีมากเลยค่ะ ขอบคุณค่ะ',
 'ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาได้เร็วมากๆเลย เป็นมิตรด้วยคับ ค่อยให้คำปรึกษาดีและรู้จักแนะนำคำปรึกษาได้ดีเช่นกันครับ ทุกเรื่องตอบได้ดีมากครับ ไม่ว่าจะเป็นโรคอื่นที่ตัวผมไม่ได้เป็นหมอก้อตอบดีมากคับแล้วคำตอบที่ตอบก้อตอบแบบมีเหตุมีผลด้วยคับ

In [15]:
sirin = pd.read_csv("comment_sirindhorn-hospital.csv")
print(sirin.shape)
sirin.head()

(283, 2)


,comment,score
0,แผนกฉุกเฉิน แผนกนรีเวชดีเยี่ยมากๆ และแผนกTBก็เ...,4
1,คนรอเยอะมากๆค่ะ (ไปตอน22.00 แผนกฉุกเฉิน) หมอคน...,4
2,ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาไ...,5
3,ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาไ...,4
4,เป็นโรงบาลที่บริการดี หมอเป็นกันเองมากครับ ทำง...,5


In [0]:
'''
gt = Translator()
input_text = "หิวข้าวรึยัง"
gt.translate(input_text, src="th", dest="en").text
'''

In [0]:
def th2en(comment):
  print(comment + "\n")
  return Translator().translate(comment, src="th", dest="en").text

In [16]:
#for check lenght with issue comment
i = 0
for x in sirin['comment']:
 if len(sirin['comment'][i]) > 1000:
  print(len(sirin['comment'][i]), "-> index : ", i)
 i=i+1

6534 -> index :  14


In [17]:
'''
There is an unexpected issue at comment 15th  - index 14
so try to remove one and run above again
'''
print(sirin.shape)
sirin.drop(14, inplace=True)
print(sirin.shape)
sirin.head()

(283, 2)
(282, 2)


,comment,score
0,แผนกฉุกเฉิน แผนกนรีเวชดีเยี่ยมากๆ และแผนกTBก็เ...,4
1,คนรอเยอะมากๆค่ะ (ไปตอน22.00 แผนกฉุกเฉิน) หมอคน...,4
2,ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาไ...,5
3,ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาไ...,4
4,เป็นโรงบาลที่บริการดี หมอเป็นกันเองมากครับ ทำง...,5


In [18]:
tqdm.pandas()

sirin["en"] = sirin.progress_apply(lambda x: th2en(x["comment"]), axis=1)

  1%|          | 2/282 [00:00<00:21, 13.28it/s]

แผนกฉุกเฉิน แผนกนรีเวชดีเยี่ยมากๆ และแผนกTBก็เยี่ยมมากๆ พนักงานธุรการไม่ค่อยโอเคเท่าไหร่แต่พยาบาลและคุณหมอดีมากๆ และดีทุกคนเลยค่ะ คุณหมอเก่งๆหลายท่านด้วยค่ะ ช่วงเช้าคนจะเยอะมากๆ อาจมีล่าช้าบ้างค่ะ แต่บางครั้งเราไป ไม่เกินเที่ยงก็ได้กลับบ้านแล้วนะคะ ถือว่าเร็วไวมากๆ ใช้บริการหลายครั้งค่อนข้างจะประทับใจค่ะ ถึงแม้จะใช้สิทธิ์ประกัน แต่ก็ได้รับการใส่ใจที่ดี ร้านอาหารร้านขนมมีให้ซื้อในราคาทั่วไปนะคะ ห้องน้ำชั้น2สะอาดดีค่ะ

คนรอเยอะมากๆค่ะ (ไปตอน22.00 แผนกฉุกเฉิน) หมอคนเดียว 
พยาบาลพูดจาดีค่ะ 
ให้บริการรักษาดี ถึงแม้จะมีหมอเพียงคนเดียว 
ราคาไม่แพง

ต่อมาไปตามนัดหมอตอนเช้า 8.00 หมอมา10.30น. คนรอกว่า 30 คนค่ะ เรียกเข้าช้ามา ผ่านไป1ชั่วโมงเพิ่งผ่านไป6คิวค่ะ หากว่าสามารถมีแนวทางในการรักษาให้รวดเร็วขึ้นจะดีมากเลยค่ะ ขอบคุณค่ะ



  1%|          | 3/282 [00:00<00:26, 10.63it/s]

ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาได้เร็วมากๆเลย เป็นมิตรด้วยคับ ค่อยให้คำปรึกษาดีและรู้จักแนะนำคำปรึกษาได้ดีเช่นกันครับ ทุกเรื่องตอบได้ดีมากครับ ไม่ว่าจะเป็นโรคอื่นที่ตัวผมไม่ได้เป็นหมอก้อตอบดีมากคับแล้วคำตอบที่ตอบก้อตอบแบบมีเหตุมีผลด้วยคับ ถือว่าเป็นอีกโรงพยาบาลที่ควรจะแนะนำให้ปรึกษาเป็นอย่างยิ่ง



  2%|▏         | 5/282 [00:00<00:35,  7.85it/s]

...

100%|█████████▉| 281/282 [03:00<00:00,  1.14it/s]

คนไข้เยอะ รอนาน แต่แพทย์ตรวจละเอียด เวลาเป็นเยอะไปหาก็หายทุกครั้ง



100%|██████████| 282/282 [03:01<00:00,  1.22it/s]

สวัสดีค่ะขอรีวิวแบ่งปันประสบการณ์นะคะ เข้ารักษาใช้บัตรทองที่โรงพยาบาลสิรินทรค่ะ พอดีตอนนี้ไม่ได้ทำงานแล้วเลยใช้สิทธิ์บัตรทอง ก่อนหน้านี้เคยได้รับการรักษามาหลายโรงพยาบาลและเคยเข้ารับการรักษาที่โรงพยาบาลมีชื่อเสียงแห่งหนึ่งแถวถนนสุขุมวิท ตอนนี้นแอดมิดไปตอนกลางคืนด้วยอสการหัวใจเต้นผิดปกติแต่คุณหมอที่อยุ่เวรจับฉีดยาอย่างเดียวบอกว่าเครียดแล้วให้ยามาทานไม่ได้ตรวจละเอียด แล้วให้กลับบ้านพอดีมีคนรุ้จักแนะนำให้ไปตรวจอาการให้ละเอียดที่โรงพยาบาลสิรินทรเลยลองไปดุ โรงพยาบาลนี้ต้องรอคิวนานค่ะเพราะเป้นโรงพยาลรัฐแต่การตรวจและรักษาละเอียดดีค่ะไม่แบ่งแยกคนไข้ให้ความเสมอภาคเท่ากัน ตรวจจนกระทั่งทราบสาเหตุที่แท้จริงของการป่วย และโรงพยาลนี้ไม่ใช่โรงพยาลของคนจนเหมือนคำพุดของบางท่านที่พุดถึงโรงพยาบาลนี้ค่ะ เพราะมีคนไข้หลายๆท่านที่ไปจ่ายเองด้วยค่ะ



100%|██████████| 282/282 [03:03<00:00,  1.53it/s]


In [19]:
sirin.head(10)

,comment,score,en
0,แผนกฉุกเฉิน แผนกนรีเวชดีเยี่ยมากๆ และแผนกTBก็เ...,4,Emergency department gynecological good tunes ...
1,คนรอเยอะมากๆค่ะ (ไปตอน22.00 แผนกฉุกเฉิน) หมอคน...,4,People waiting for a lot of it (go to the emer...
2,ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาไ...,5,Service is very good tight The problem is the ...
3,ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาไ...,4,Service is very good tight The problem is the ...
4,เป็นโรงบาลที่บริการดี หมอเป็นกันเองมากครับ ทำง...,5,"The hospital is good I am very friendly, fast ..."
5,ดีมากๆครับ หมอรักษาด้วยการให้บำรุง...,5,Very good doctors to treat with care because o...
6,ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาไ...,5,Service is very good tight The problem is the ...
7,ดีมากๆครับ หมอรักษาด้วยการให้บำรุงเพราะเป็นโรค...,4,Very good doctor with the treatment because of...
8,เป็น. รพ. ที่ดีมากน้ะคะ บริการดีเราถามอะไรทั้ง...,5,"A. Hospital, very good to me. Good service, we..."
9,พยาบาลดูแลและบริการดีส่วนคุณหมอก็บริการดีมากเล...,5,Nursing care services and good medical service...


In [21]:
rename_col = ['comment','en','score']
sirin = sirin.reindex(columns=rename_col)
sirin.columns = ['th_comment','en_comment','score']
sirin.head()

,th_comment,en_comment,score
0,แผนกฉุกเฉิน แผนกนรีเวชดีเยี่ยมากๆ และแผนกTBก็เ...,Emergency department gynecological good tunes ...,4
1,คนรอเยอะมากๆค่ะ (ไปตอน22.00 แผนกฉุกเฉิน) หมอคน...,People waiting for a lot of it (go to the emer...,4
2,ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาไ...,Service is very good tight The problem is the ...,5
3,ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาไ...,Service is very good tight The problem is the ...,4
4,เป็นโรงบาลที่บริการดี หมอเป็นกันเองมากครับ ทำง...,"The hospital is good I am very friendly, fast ...",5


In [0]:
#export to csv incluing th-comment, en-comment, score
sirin.to_csv('en-comment_sirindhorn-hospital.csv', index=False ,encoding='utf-8')

In [23]:
en_sirin = pd.read_csv("en-comment_sirindhorn-hospital.csv")
print(en_sirin.shape)
en_sirin.head()

(282, 3)


,th_comment,en_comment,score
0,แผนกฉุกเฉิน แผนกนรีเวชดีเยี่ยมากๆ และแผนกTBก็เ...,Emergency department gynecological good tunes ...,4
1,คนรอเยอะมากๆค่ะ (ไปตอน22.00 แผนกฉุกเฉิน) หมอคน...,People waiting for a lot of it (go to the emer...,4
2,ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาไ...,Service is very good tight The problem is the ...,5
3,ให้บริการดีมากเลยคับ เป็นโรงพยาบาลที่ตอบปัญหาไ...,Service is very good tight The problem is the ...,4
4,เป็นโรงบาลที่บริการดี หมอเป็นกันเองมากครับ ทำง...,"The hospital is good I am very friendly, fast ...",5
